In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import data_loader

### setup

In [25]:
xs, ys = data_loader.verified_subjects_calibrated_yprs(resampled=True, flatten=True)
xs = np.array(xs)
ys = np.array(ys)
trainx, testx, trainy, testy = train_test_split(xs, ys, test_size=0.2)

haobin_11_22
kevin_11_7
russell_11_7
kelly_11_7
russell_11_20_stand


In [36]:
train_subjects = data_loader.VERIFIED_SUBJECTS[:-1]
test_subjects = data_loader.VERIFIED_SUBJECTS[-1:]
train_subjects , test_subjects

(['haobin_11_22', 'kevin_11_7', 'russell_11_7', 'kelly_11_7'],
 ['russell_11_20_stand'])

In [37]:
trainx, trainy = data_loader.verified_subjects_calibrated_yprs(resampled=True, flatten=True, subjects=train_subjects)
testx, testy = data_loader.verified_subjects_calibrated_yprs(resampled=True, flatten=True, subjects=test_subjects)
trainx = np.array(trainx)
trainy = np.array(trainy)
testx = np.array(testx)
testy = np.array(testy)

haobin_11_22
kevin_11_7
russell_11_7
kelly_11_7
russell_11_20_stand


In [38]:
trainx.shape, trainy.shape, testx.shape, testy.shape

((2090, 300), (2090,), (520, 300), (520,))

In [39]:
def get_acc(pred, testy):
    correct_pred = (pred-testy == 0).astype(int)
    acc = np.sum(correct_pred) / pred.shape[0]
    return acc

### SVM

In [40]:
from sklearn import svm

In [41]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svc_acc = {}

for kernel in kernels:
    print(f'Training svc with {kernel} kernel')
    clf = svm.SVC(kernel=kernel, gamma='auto')
    clf.fit(trainx, trainy)
    pred = clf.predict(testx)
    svc_acc[kernel] = get_acc(pred, testy)

Training svc with linear kernel
Training svc with poly kernel
Training svc with rbf kernel
Training svc with sigmoid kernel


In [42]:
svc_acc

{'linear': 0.2403846153846154,
 'poly': 0.25576923076923075,
 'rbf': 0.038461538461538464,
 'sigmoid': 0.038461538461538464}

### KNN

In [43]:
from sklearn.neighbors import KNeighborsClassifier

In [44]:
knn_acc = {}

for num_neighbor in range(2, 15):
    print(f'Running KNN with n={num_neighbor}')
    
    clf = KNeighborsClassifier(n_neighbors=num_neighbor)
    clf.fit(trainx, trainy)
    
    pred = clf.predict(testx)
    knn_acc[num_neighbor] = get_acc(pred, testy)

Running KNN with n=2
Running KNN with n=3
Running KNN with n=4
Running KNN with n=5
Running KNN with n=6
Running KNN with n=7
Running KNN with n=8
Running KNN with n=9
Running KNN with n=10
Running KNN with n=11
Running KNN with n=12
Running KNN with n=13
Running KNN with n=14


In [45]:
knn_acc

{2: 0.2423076923076923,
 3: 0.24423076923076922,
 4: 0.24423076923076922,
 5: 0.2230769230769231,
 6: 0.2173076923076923,
 7: 0.2153846153846154,
 8: 0.21346153846153845,
 9: 0.21923076923076923,
 10: 0.2173076923076923,
 11: 0.19615384615384615,
 12: 0.19807692307692307,
 13: 0.2,
 14: 0.19423076923076923}

### Nearest Centroid

In [16]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

In [17]:
clf = NearestCentroid()
clf.fit(trainx, trainy)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [18]:
pred = clf.predict(testx)
correct_pred = (pred-testy == 0).astype(int)
acc = np.sum(correct_pred) / pred.shape[0]
acc

0.05471698113207547

### Lasso: linear regression

In [19]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1, max_iter=5000)
clf.fit(trainx, trainy)
pred = clf.predict(testx)
pred = np.rint(pred)
acc = get_acc(pred, testy)

In [20]:
acc

0.04905660377358491

### Gaussian Naive Bayes

In [21]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(trainx, trainy)
pred = clf.predict(testx)
acc = get_acc(pred, testy)

In [22]:
acc

0.04339622641509434

### Forest of randomized trees

In [23]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trainx, trainy)
pred = clf.predict(testx)
acc = get_acc(pred, testy)

/Users/junshenchen/anaconda3/envs/cs229proj-mac/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [24]:
acc

0.09433962264150944

### Basic NN (multi-layer perceptron)

In [23]:
from sklearn.neural_network import MLPClassifier

In [25]:
structures = [
    (100,50),
    (100,100,50),
    (200,50),
    (200,100,50),
]

activations = ['logistic', 'tanh', 'relu']

l2_reg_const = [0.0001 * x for x in [1, 5, 10]]

nn_acc = {}

In [27]:
for structure in structures:
    for act in activations:
        for alpha in l2_reg_const:
            name = f'nn-{str(structure)}-{act}-{alpha}'
            print(name)
            
            clf = MLPClassifier(
                hidden_layer_sizes=structure,
                activation=act,
                alpha=alpha,
                max_iter=5000
            )
            clf.fit(trainx, trainy)
            pred = clf.predict(testx)
            
            nn_acc[name] = get_acc(pred, testy)

nn-(100, 50)-logistic-0.0001
nn-(100, 50)-logistic-0.0005
nn-(100, 50)-logistic-0.001
nn-(100, 50)-tanh-0.0001
nn-(100, 50)-tanh-0.0005
nn-(100, 50)-tanh-0.001
nn-(100, 50)-relu-0.0001
nn-(100, 50)-relu-0.0005
nn-(100, 50)-relu-0.001
nn-(100, 100, 50)-logistic-0.0001
nn-(100, 100, 50)-logistic-0.0005
nn-(100, 100, 50)-logistic-0.001
nn-(100, 100, 50)-tanh-0.0001
nn-(100, 100, 50)-tanh-0.0005
nn-(100, 100, 50)-tanh-0.001
nn-(100, 100, 50)-relu-0.0001
nn-(100, 100, 50)-relu-0.0005
nn-(100, 100, 50)-relu-0.001
nn-(200, 50)-logistic-0.0001
nn-(200, 50)-logistic-0.0005
nn-(200, 50)-logistic-0.001
nn-(200, 50)-tanh-0.0001
nn-(200, 50)-tanh-0.0005
nn-(200, 50)-tanh-0.001
nn-(200, 50)-relu-0.0001
nn-(200, 50)-relu-0.0005
nn-(200, 50)-relu-0.001
nn-(200, 100, 50)-logistic-0.0001
nn-(200, 100, 50)-logistic-0.0005
nn-(200, 100, 50)-logistic-0.001
nn-(200, 100, 50)-tanh-0.0001
nn-(200, 100, 50)-tanh-0.0005
nn-(200, 100, 50)-tanh-0.001
nn-(200, 100, 50)-relu-0.0001
nn-(200, 100, 50)-relu-0.0005
nn-

In [28]:
nn_acc

{'nn-(100, 50)-logistic-0.0001': 0.6800766283524904,
 'nn-(100, 50)-logistic-0.0005': 0.6666666666666666,
 'nn-(100, 50)-logistic-0.001': 0.6685823754789272,
 'nn-(100, 50)-tanh-0.0001': 0.6513409961685823,
 'nn-(100, 50)-tanh-0.0005': 0.6666666666666666,
 'nn-(100, 50)-tanh-0.001': 0.6954022988505747,
 'nn-(100, 50)-relu-0.0001': 0.7298850574712644,
 'nn-(100, 50)-relu-0.0005': 0.7203065134099617,
 'nn-(100, 50)-relu-0.001': 0.7260536398467433,
 'nn-(100, 100, 50)-logistic-0.0001': 0.5363984674329502,
 'nn-(100, 100, 50)-logistic-0.0005': 0.5766283524904214,
 'nn-(100, 100, 50)-logistic-0.001': 0.5478927203065134,
 'nn-(100, 100, 50)-tanh-0.0001': 0.6877394636015326,
 'nn-(100, 100, 50)-tanh-0.0005': 0.6915708812260536,
 'nn-(100, 100, 50)-tanh-0.001': 0.6800766283524904,
 'nn-(100, 100, 50)-relu-0.0001': 0.7777777777777778,
 'nn-(100, 100, 50)-relu-0.0005': 0.7911877394636015,
 'nn-(100, 100, 50)-relu-0.001': 0.7509578544061303,
 'nn-(200, 50)-logistic-0.0001': 0.7528735632183908,
 '